In [25]:

# coding: utf-8
import pandas as pd
# These lines set up the plotting funstionality and formatting.
import matplotlib 
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
matplotlib.use('Agg', warn = False)
#get_ipython().run_line_magic('matplotlib', 'inline')

import numpy as np


from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.model_selection import train_test_split

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import itertools

import h2o
from h2o.automl import H2OAutoML

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# 1. Load data

In [43]:
df = pd.read_pickle("processed_data.pickle")
p_grade_map={}
j=0
for i in pd.unique(df['p_grade']):
    p_grade_map[i]=j
    j=j+1
benefit_map={}
j=0
for i in pd.unique(df['benefit']):
    benefit_map[i]=j
    j=j+1
e_type_map={}
j=0
for i in pd.unique(df['e_type']):
    e_type_map[i]=j
    j=j+1
    
df['p_grade']=df['p_grade'].map(p_grade_map)
df['benefit']=df['benefit'].map(benefit_map)
df['e_type']=df['e_type'].map(e_type_map)

In [99]:
df.head()

,Total Payments,Lump Sum Pay,Other Pay (Payroll Explorer),FMS Department,Job Class,Average Benefit Cost,Time,p_dep,e_type,Rate,p_bonus,t_bonus,o_pay,long,e_pay,p_grade,insurance,benefit,financial,Salary
161542,4847.83,0.0,1740.73,98,941,18755.63,3,4594.945881,0,31.29,0.00,0.0,0.00,0.00,1740.73,0,17486.94,0,98,3107.10
167941,56481.08,0.0,3480.12,14,1358,11435.16,3,1603.000000,0,26.47,2869.48,0.0,1440.96,610.64,0.00,1,11012.40,1,14,51560.00
244088,138165.59,0.0,4609.98,98,3115,19392.62,5,4594.945881,0,38.26,0.00,0.0,55497.61,0.00,4609.98,0,18144.43,0,98,78058.00
15392,111777.71,0.0,14526.10,70,2214,12740.52,1,4301.000000,0,52.66,13293.10,1233.0,2621.55,0.00,0.00,2,11651.40,2,70,94630.06
69682,9961.87,0.0,565.20,88,2498,0.00,2,7903.000000,1,14.79,0.00,304.0,0.00,261.20,0.00,3,0.00,3,88,9396.67


# 2. Data spliting into test and train

In [53]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:19], df.iloc[:,19], test_size=0.33, random_state=42)


In [54]:
train = pd.concat([X_train, y_train], axis=1,join_axes=[X_train.index])
test = pd.concat([X_test, y_test], axis = 1, join_axes = [X_test.index])

# 3. use H2O to do automated model building and parameter training

In [67]:
h2oTest = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [61]:
h2oTrain = h2o.H2OFrame(train)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [62]:
aml=H2OAutoML(max_models=15, seed = 1)
aml.train(x = h2oTrain.columns, y = "Salary", training_frame=h2oTrain )

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [63]:
lb = aml.leaderboard
lb.head(rows = lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20181030_220731,3.10973e+06,1763.44,3.10973e+06,728.655,nan
XGBoost_2_AutoML_20181030_220731,4.38603e+06,2094.29,4.38603e+06,848.682,nan
DRF_1_AutoML_20181030_220731,8.99423e+06,2999.04,8.99423e+06,1128.01,nan
XRT_1_AutoML_20181030_220731,9.51055e+06,3083.92,9.51055e+06,1164.57,nan
StackedEnsemble_AllModels_AutoML_20181030_220731,1.7213e+09,41488.5,1.7213e+09,35363.9,nan
StackedEnsemble_BestOfFamily_AutoML_20181030_220731,1.7213e+09,41488.5,1.7213e+09,35363.9,nan
GLM_grid_1_AutoML_20181030_220731_model_1,1.72149e+09,41490.8,1.72149e+09,35366.3,nan


In [64]:
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20181030_220731


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 531213.8801756738
RMSE: 728.8442084394125
MAE: 497.3353064600044
RMSLE: NaN
Mean Residual Deviance: 531213.8801756738

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 1939439.327721194
RMSE: 1392.637543555822
MAE: 678.8022749948734
RMSLE: NaN
Mean Residual Deviance: 1939439.327721194

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 3109730.9267370617
RMSE: 1763.4429184799437
MAE: 728.6546719727695
RMSLE: NaN
Mean Residual Deviance: 3109730.9267370617
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,728.65375,60.064953,773.3274,649.47095,658.5507,874.3034,687.6163
mean_residual_deviance,3109717.0,1235499.6,3778475.8,1727868.9,1933630.9,6261133.0,1847477.1
mse,3109717.0,1235499.6,3778475.8,1727868.9,1933630.9,6261133.0,1847477.1
r2,0.9982011,0.0007069,0.9978619,0.9989891,0.9988551,0.9963798,0.9989195
residual_deviance,3109717.0,1235499.6,3778475.8,1727868.9,1933630.9,6261133.0,1847477.1
rmse,1702.062,326.1151,1943.8302,1314.4843,1390.5505,2502.2256,1359.2194
rmsle,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2018-10-30 22:40:13,30 min 51.008 sec,0.0,72447.8134686,59392.0007194,5248685676.3855286,72495.4886827,59754.3842357,5255595879.3415165
,2018-10-30 22:40:13,30 min 51.312 sec,5.0,56431.7176825,46005.5263890,3184538760.5925298,56406.7315909,46248.7659106,3181719368.7705655
,2018-10-30 22:40:13,30 min 51.535 sec,10.0,44119.5636401,35708.8589964,1946535895.7932498,44019.5602869,35847.9667381,1937721687.8522286
,2018-10-30 22:40:14,30 min 51.770 sec,15.0,34504.4176925,27715.4676456,1190554840.2956963,34367.4822093,27797.9120315,1181123833.4054093
,2018-10-30 22:40:14,30 min 52.023 sec,20.0,27325.4671743,21641.6993861,746681156.2910906,27159.0086263,21682.6530802,737611749.5619690
---,---,---,---,---,---,---,---,---,---
,2018-10-30 22:43:59,34 min 37.010 sec,1230.0,734.4821317,500.8813934,539464.0018341,1396.3577163,681.8657737,1949814.8717864
,2018-10-30 22:44:01,34 min 38.623 sec,1235.0,732.1995698,499.3579022,536116.2099933,1394.7643938,680.6148933,1945367.7141404
,2018-10-30 22:44:02,34 min 40.236 sec,1240.0,730.8646197,498.6001807,534163.0923243,1393.7303249,680.0173471,1942484.2186590
,2018-10-30 22:44:04,34 min 42.124 sec,1245.0,729.4381466,497.6679271,532080.0096968,1392.9340048,679.0637585,1940265.1418604



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
TotalPayments,6764.0,1.0,0.2015675
Rate,5468.0,0.8083974,0.1629466
o_pay,4003.0,0.5918096,0.1192896
OtherPay(PayrollExplorer),3799.0,0.5616499,0.1132104
e_pay,2248.0,0.3323477,0.0669905
JobClass,2152.0,0.3181549,0.0641297
LumpSumPay,1477.0,0.2183619,0.0440147
p_bonus,1396.0,0.2063868,0.0416009
AverageBenefitCost,1237.0,0.1828800,0.0368627
Time,1234.0,0.1824364,0.0367733


# 4. Save the Xgboost Model: 

In [97]:
#save the model:
model_path = h2o.save_model(model = aml.leader, path = "./",force = True)

In [101]:
print(model_path)

/Users/xinyuzhang/Desktop/hackthon_gwu/XGBoost_1_AutoML_20181030_220731


In [102]:
type(model_path)

str

# 5. Prediction and output

In [75]:
preds = aml.leader.predict(h2oTest)
preds = preds.as_data_frame()

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [92]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, preds)

2628849.1337822997

In [100]:
print(aml.leader)

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20181030_220731


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 531213.8801756738
RMSE: 728.8442084394125
MAE: 497.3353064600044
RMSLE: NaN
Mean Residual Deviance: 531213.8801756738

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 1939439.327721194
RMSE: 1392.637543555822
MAE: 678.8022749948734
RMSLE: NaN
Mean Residual Deviance: 1939439.327721194

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 3109730.9267370617
RMSE: 1763.4429184799437
MAE: 728.6546719727695
RMSLE: NaN
Mean Residual Deviance: 3109730.9267370617
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,728.65375,60.064953,773.3274,649.47095,658.5507,874.3034,687.6163
mean_residual_deviance,3109717.0,1235499.6,3778475.8,1727868.9,1933630.9,6261133.0,1847477.1
mse,3109717.0,1235499.6,3778475.8,1727868.9,1933630.9,6261133.0,1847477.1
r2,0.9982011,0.0007069,0.9978619,0.9989891,0.9988551,0.9963798,0.9989195
residual_deviance,3109717.0,1235499.6,3778475.8,1727868.9,1933630.9,6261133.0,1847477.1
rmse,1702.062,326.1151,1943.8302,1314.4843,1390.5505,2502.2256,1359.2194
rmsle,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2018-10-30 22:40:13,30 min 51.008 sec,0.0,72447.8134686,59392.0007194,5248685676.3855286,72495.4886827,59754.3842357,5255595879.3415165
,2018-10-30 22:40:13,30 min 51.312 sec,5.0,56431.7176825,46005.5263890,3184538760.5925298,56406.7315909,46248.7659106,3181719368.7705655
,2018-10-30 22:40:13,30 min 51.535 sec,10.0,44119.5636401,35708.8589964,1946535895.7932498,44019.5602869,35847.9667381,1937721687.8522286
,2018-10-30 22:40:14,30 min 51.770 sec,15.0,34504.4176925,27715.4676456,1190554840.2956963,34367.4822093,27797.9120315,1181123833.4054093
,2018-10-30 22:40:14,30 min 52.023 sec,20.0,27325.4671743,21641.6993861,746681156.2910906,27159.0086263,21682.6530802,737611749.5619690
---,---,---,---,---,---,---,---,---,---
,2018-10-30 22:43:59,34 min 37.010 sec,1230.0,734.4821317,500.8813934,539464.0018341,1396.3577163,681.8657737,1949814.8717864
,2018-10-30 22:44:01,34 min 38.623 sec,1235.0,732.1995698,499.3579022,536116.2099933,1394.7643938,680.6148933,1945367.7141404
,2018-10-30 22:44:02,34 min 40.236 sec,1240.0,730.8646197,498.6001807,534163.0923243,1393.7303249,680.0173471,1942484.2186590
,2018-10-30 22:44:04,34 min 42.124 sec,1245.0,729.4381466,497.6679271,532080.0096968,1392.9340048,679.0637585,1940265.1418604



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
TotalPayments,6764.0,1.0,0.2015675
Rate,5468.0,0.8083974,0.1629466
o_pay,4003.0,0.5918096,0.1192896
OtherPay(PayrollExplorer),3799.0,0.5616499,0.1132104
e_pay,2248.0,0.3323477,0.0669905
JobClass,2152.0,0.3181549,0.0641297
LumpSumPay,1477.0,0.2183619,0.0440147
p_bonus,1396.0,0.2063868,0.0416009
AverageBenefitCost,1237.0,0.1828800,0.0368627
Time,1234.0,0.1824364,0.0367733


In [111]:
aml.leader.model_performance(h2oTest)


ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 2628849.025600663
RMSE: 1621.3725745801496
MAE: 691.7537353697554
RMSLE: NaN
Mean Residual Deviance: 2628849.025600663
